In [13]:
import json

In [14]:
with open("../DataProcessing/TestData/postGame.json", "r", encoding="utf-8") as file:
    postData = json.load(file)

In [24]:
def filterPostData(data):
    # Define the keys to keep in the info section using camelCase
    infoKeysToKeep = {'endOfGameResult', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId',
                      'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId',
                      'platformId', 'queueId', 'teams', 'tournamentCode'}
    
    # Define participant keys to keep, adjusted to camelCase as necessary
    participantKeysToKeep = {'championName', 'damageDealtToBuildings', 'damageDealtToObjectives',
                             'damageDealtToTurrets', 'goldEarned', 'goldSpent', 'individualPosition',
                             'inhibitorTakedowns', 'lane', 'participantId', 'perks', 'puuid', 'riotIdGameName',
                             'riotIdTagline', 'role', 'summonerId', 'summonerName', 'teamEarlySurrendered', 'teamId',
                             'teamPosition', 'totalAllyJungleMinionsKilled', 'totalDamageDealt',
                             'totalDamageDealtToChampions', 'totalDamageTaken', 'totalEnemyJungleMinionsKilled',
                             'turretTakedowns', 'visionScore', 'wardsKilled', 'wardsPlaced', 'win', 'challenges'}

    # Define the keys to keep within the challenges dictionary
    challengesKeysToKeep = {'controlWardsPlaced', 'dragonTakedowns', 'goldPerMinute', 'kda',
                            'killParticipation', 'killsNearEnemyTurret', 'killsUnderOwnTurret',
                            'maxCsAdvantageOnLaneOpponent', 'maxLevelLeadLaneOpponent', 'soloKills',
                            'stealthWardsPlaced', 'takedownOnFirstTurret', 'takedowns', 'teamBaronKills',
                            'teamDamagePercentage', 'teamElderDragonKills', 'teamRiftHeraldKills',
                            'visionScorePerMinute', 'wardTakedowns'}
    
    # Keep entire metadata
    filteredData = {'metadata': data['metadata']}
    
    # Filter info section
    filteredInfo = {key: data['info'][key] for key in data['info'] if key in infoKeysToKeep}
    
    # Filter participants
    filteredParticipants = []
    for participant in data['info']['participants']:
        filteredParticipant = {}
        for key in participant:
            if key in participantKeysToKeep:
                if key == 'challenges':
                    # Special handling for the challenges dictionary
                    challenges = participant[key]
                    filteredChallenges = {ckey: challenges[ckey] for ckey in challenges if ckey in challengesKeysToKeep}
                    filteredParticipant[key] = filteredChallenges
                else:
                    filteredParticipant[key] = participant[key]
        filteredParticipants.append(filteredParticipant)
    
    # Add filtered info and participants to the filteredData
    filteredInfo['participants'] = filteredParticipants
    filteredData['info'] = filteredInfo

    return filteredData


# Filter the dataset
filteredPostData = filterPostData(postData)

filePath = '../DataProcessing/TestData/filteredPostGameData.json'

# Save the filtered dataset back to a file
with open(filePath, 'w') as file:
    json.dump(filteredPostData, file, indent=4)

print("Filtered post game data saved.")

Filtered post game data saved.


In [16]:
def getTakedowns(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            takedowns = participant.get('challenges', {}).get('takedowns', None)
            if takedowns is not None:
                print(f"  Takedowns by {targetChampionName}: {takedowns}")
            else:
                print(f"No Data for Takedowns by {targetChampionName}")
    return None

def getKda(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            kda = participant.get('challenges', {}).get('kda', None)
            if kda is not None:
                print(f"  KDA by {targetChampionName}: {kda}\n")
            else:
                print(f"No KDA found for {targetChampionName}\n")
    return None

def getSoloKills(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            soloKills = participant.get('challenges', {}).get('soloKills', None)
            if soloKills is not None:
                print(f"  Solo Kills for {targetChampionName}: {soloKills}")
            else:
                print(f"No kill participation found for {targetChampionName}")
    return None
    
def getKillParticipation(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            killParticipation = participant.get('challenges', {}).get('killParticipation', None)
            if killParticipation is not None:
                print(f"  Kill Participation for {targetChampionName}: {killParticipation}")
            else:
                print(f"No kill participation found for {targetChampionName}")
            return killParticipation
    return None    

def getLaneRole(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            laneRole = participant.get('individualPosition', None)
            if laneRole is not None:
                print(f"Role of {targetChampionName}: {laneRole}")
            return laneRole
    return None  
    
def getJungleStats(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid and participant.get('individualPosition') == 'JUNGLE':
            totalAllyJungleMinionsKilled = participant.get('totalAllyJungleMinionsKilled', None)
            totalEnemyJungleMinionsKilled = participant.get('totalEnemyJungleMinionsKilled', None)
            if totalAllyJungleMinionsKilled is not None and totalEnemyJungleMinionsKilled is not None:
                print(f"  Total Ally Jungle Minions Killed: {totalAllyJungleMinionsKilled}")
                print(f"  Total Enemy Jungle Minions Killed: {totalEnemyJungleMinionsKilled}\n")
            return totalAllyJungleMinionsKilled, totalEnemyJungleMinionsKilled
    return None, None

def getTeamSide(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamSide = participant.get('teamId', None)
            if teamSide is not None:
                if teamSide == 100:
                    print(f"{targetChampionName} was Blue Side")
                else:
                    print(f"{targetChampionName} was Red Side")
            return teamSide
    return None

def getGameWon(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            gameWon = participant.get('win', None)
            if gameWon is not None:
                if gameWon == True:
                    print(f"{targetChampionName}s Team Won the Game")
                else:
                    print(f"{targetChampionName}s Team Lost the Game")        
            return
    return None

def getTotalDamageDealt(postfilteredPostDataData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageDealt = participant.get('totalDamageDealt', None)
            if totalDamageDealt is not None:
                print(f"  Total Damage Dealt by {targetChampionName}: {totalDamageDealt}")
    return None

def getTotalDamageDealtToChampions(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageDealtToChampions = participant.get('totalDamageDealtToChampions', None)
            if totalDamageDealtToChampions is not None:
                print(f"  Total Damage Dealt To Champions by {targetChampionName}: {totalDamageDealtToChampions}")
    return None

def getTotalDamageTaken(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageTaken = participant.get('totalDamageTaken', None)
            if totalDamageTaken is not None:
                print(f"  Total Damage Taken for {targetChampionName}: {totalDamageTaken}\n")
    return None

def getDeathsByEnemyChamps(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            deathsByEnemyChamps =  participant.get('challenges', {}).get('deathsByEnemyChamps', None)
            if deathsByEnemyChamps is not None:
                print(f"  Deaths by Enemy Champions on {targetChampionName}: {deathsByEnemyChamps}")
    return None

def getTeamBaronKills(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamBaronKills = participant.get('challenges', {}).get('teamBaronKills', None)
            if teamBaronKills is not None and teamBaronKills != 0:
                print(f"  Team Baron Kills by {targetChampionName}: {teamBaronKills}")
    return None

def getTeamRiftHeraldKills(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamRiftHeraldKills = participant.get('challenges', {}).get('teamRiftHeraldKills', None)
            if teamRiftHeraldKills is not None and teamRiftHeraldKills != 0:
                print(f"  Team Rift Herald Kills by {targetChampionName}: {teamRiftHeraldKills}")
    return None

def getTeamElderDragonKills(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamElderDragonKills = participant.get('challenges', {}).get('teamElderDragonKills', None)
            if teamElderDragonKills is not None and teamElderDragonKills != 0:
                print(f"  Team Elder Dragon Kills by {targetChampionName}: {teamElderDragonKills}")
    return None

def getDragonTakedowns(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            dragonTakedowns = participant.get('challenges', {}).get('dragonTakedowns', None)
            if dragonTakedowns is not None and dragonTakedowns != 0:
                print(f"  Dragon Takedowns by {targetChampionName}: {dragonTakedowns}")
    return None

def getVisionScoreAdvantageLaneOpponent(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            visionScoreAdvantage = participant.get('challenges', {}).get('visionScoreAdvantageLaneOpponent')
            if visionScoreAdvantage is not None:
                print(f"  Vision Score Advantage for {targetChampionName}: {visionScoreAdvantage}")
    return None

def getTurretTakedowns(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            turretTakedowns = participant.get('challenges', {}).get('turretTakedowns', None)
            if turretTakedowns is not None and turretTakedowns != 0:
                print(f"  Turret Takedowns by {targetChampionName}: {turretTakedowns}\n")  
    return None

def getVisionScore(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            visionScore = participant.get('visionScore', None)
            if visionScore is not None:
                print(f"  Vison Score for {targetChampionName}: {visionScore}")
    return None

def getControlWardsPlaced(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            controlWardsPlaced = participant.get('challenges', {}).get('controlWardsPlaced', None)
            if controlWardsPlaced is not None and controlWardsPlaced != 0:
                print(f"  Control Wards placed by {targetChampionName}: {controlWardsPlaced}")
    return None

def getStealthWardsPlaced(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            stealthWardsPlaced = participant.get('challenges', {}).get('stealthWardsPlaced', None)
            if stealthWardsPlaced is not None and stealthWardsPlaced != 0:
                print(f"  Stealth Wards Placed by {targetChampionName}: {stealthWardsPlaced}")
    return None

def getWardsKilled(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            wardsKilled = participant.get('wardsKilled', None)
            if wardsKilled is not None and wardsKilled != 0:
                print(f"  Wards Killed by {targetChampionName}: {wardsKilled}\n")
    return None

def getWardsPlaced(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            wardsPlaced = participant.get('wardsPlaced', None)
            if wardsPlaced is not None and wardsPlaced != 0:
                print(f"  Wards Placed by {targetChampionName}: {wardsPlaced}")
    return None

def getLaneMinionsFirst10Minutes(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            laneMinionsFirst10Minutes = participant.get('challenges', {}).get('laneMinionsFirst10Minutes', None)
            if laneMinionsFirst10Minutes is not None and getLaneRole != "JUNGLE":
                print(f"  Lane Minions First 10 Minutes by {targetChampionName}: {laneMinionsFirst10Minutes}\n") 
    return None

def getGoldPerMinute(filteredPostData, targetPuuid):
    for participant in filteredPostData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            goldPerMinute = participant.get('challenges', {}).get('goldPerMinute', None)
            if goldPerMinute is not None:
                print(f"  Gold Per Minute for {targetChampionName}: {goldPerMinute}")
    return None